In [28]:
import cv2
from tqdm import tqdm
import numpy as np
from vidaug import augmentors as vidaug
import json
import matplotlib.pyplot as plt

import PIL.Image as im
import json
%matplotlibo.inline

UsageError: Line magic function `%matplotlibo.inline` not found.


In [103]:
import glob
from IPython.display import Video


def Test_Play(path='./dataset/examples/'):
    Path=sorted(glob.glob(os.path.join(path,'*.mp4')))
    Video(Path[10])


#idx返す?
def keystoint(x):
    return {int(k): v for k,v in x.items()}


def get_rotate(frame,width,height,degree):
    #Affine用の行列を指定        
    rotation_mat=cv2.getRotationMatrix2D((width/2,height/2),degree,1)
    #Affine変換
    rotate_frame=cv2.warpAffine(frame,rotation_mat,(width,height))
    return rotate_frame

def get_translate(frame,width,height,translate):
    Mat=np.float32([[1,0,translate[0]],[0,1,translate[1]]])
    translate_frame=cv2.warpAffine(frame,Mat,(width,height))
    return translate_frame



def Augment_tools(path,out_dir,video_id,degree=None,translate=None):
    """
    回転はdegree 指定
    平行移動はtranslate 指定
    """
    
    cap=cv2.VideoCapture(path)
    fps=int(cap.get(cv2.CAP_PROP_FPS))
    frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    frame_width,frame_height=int(cap.get(3)),int(cap.get(4))
    fourc=cv2.VideoWriter_fourcc(*'mp4v')
    
    if degree is not None:
        save_path=out_dir+video_id+'_rotate_'+str(degree)+'.mp4'
    elif translate is not None:
        save_path=out_dir+video_id+'_translate_'+str(translate[0]) +"_" + str(translate[1]) + ".mp4"
    Video_writer=cv2.VideoWriter(save_path,fourc,fps,(frame_width,frame_height))
    
    
    while (cap.isOpened()):
        ret,frame=cap.read()
        if not ret:
            break
        if degree is not None:
            out_frame=get_rotate(frame,frame_width,frame_height,degree)
        elif translate is not None:
            out_frame=get_translate(frame,frame_width,frame_height,translate)
        Video_writer.write(out_frame)
    cap.release()
    Video_writer.release()

    
def AugmentVideos(annotation_dict,labels_dict,data_dir='./dataset/examples/',
                 out_dir='./dataset/augment-examples/'):
    """
    Annotation_dict  dict形式  'video_id':cls_label　のセット
    
    labels_dict     cls_label : action name
    
    """
    
    with open(annotation_dict) as f:
        annotation_dict=json.load(f)
        video_list=sorted(list(annotation_dict.items()))
        
    with open(labels_dict) as f:
        #object hook１を使用して　keyとvalueを返す
        labels_dict=json.load(f,object_hook=keystoint)
    
    #Visualize  each actions  numbers
    count_dict=dict()
    for key in annotation_dict:
        if labels_dict[annotation_dict[key]] in count_dict:
            count_dict[labels_dict[annotation_dict[key]]]+=1
        else:
            count_dict[labels_dict[annotation_dict[key]]]=1
            

    sorted_dict={k:v for k,v in sorted(count_dict.items(),key=lambda item:item[1])}
    filterd_list=[k for k,v in sorted_dict.items() if v <=2000]
     
    print(sorted_dict)
    
    #Augment 実施
    augmented_annotation=dict()
    pbar=tqdm(video_list)
    i=0
    for video_id,label in pbar:
        path=data_dir+video_id+'.mp4'        
        
        if labels_dict[label] in  filterd_list:
            augmented_annotation[video_id+'_rotate_30']=label
            Augment_tools(path,out_dir,video_id,degree=30)
            augmented_annotation[video_id+'_rotate_330']=label
            Augment_tools(path,out_dir,video_id,degree=330)

            augmented_annotation[video_id + "_translate_32_0"] = label
            Augment_tools(path, out_dir, video_id, translate=(32, 0))
            augmented_annotation[video_id + "_translate_-32_0"] = label
            Augment_tools(path, out_dir, video_id, translate=(-32, 0))
        i+=1
        pbar.set_description('Percentage {}'.format(i/len(video_list)))
        
        with open('./Test_dataset/augment_annotation_dict','w') as fp:
            json.dump(augmented_annotation,fp)
    
        

In [105]:
annotation_dict='./annotation_dict.json'
labels_dict='./labels_dict2.json'
data_dir='./examples/'
out_dir='../Test_dataset/augment_dataset/'
AugmentVideos(annotation_dict,labels_dict,data_dir=data_dir,out_dir=out_dir)

{'shoot': 426, 'pick': 712, 'block': 996, 'pass': 1070, 'ball in hand': 2362, 'dribble': 3490, 'defense': 3866, 'run': 5924, 'no_action': 6490, 'walk': 11749}


Percentage 1.0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 37085/37085 [05:01<00:00, 122.93it/s]


## Augment結果の確認


In [ ]:
annotation_dict='./Test_dataset/augment_annotation_dict'
labels_dict='./dataset/labels_dict2.json'
with open(annotation_dict) as f:
    annotation_dict=json.load(f)
    video_list=sorted(list(annotation_dict.items()))
with open(labels_dict) as f:
    labels_dict=json.load(f,object_hook=keystoint)
count_dict=dict()
for key in annotation_dict:
    if labels_dict[annotation_dict[key]] in count_dict:
        count_dict[labels_dict[annotation_dict[key]]]+=1
    else:
        count_dict[labels_dict[annotation_dict[key]]]=1
sorted_dict={k:v for k,v in sorted(count_dict.items(),key=lambda item:item[1])}
print(sorted_dict)

video_path=glob.glob('./Test_dataset/augment_dataset/*.mp4')
print(len(video_path))